In [186]:
import numpy as np
import pandas as pd
from kMeans import kMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import completeness_score, homogeneity_score

## 2-A
Create a cosine distance function.

In [226]:
def cosine_distance(x,y):
    x = np.asarray(x)
    y = np.asarray(y)
    x_norm = np.linalg.norm(x)
    y_norm = np.linalg.norm(y)
    if x_norm == 0 or y_norm == 0:
        return np.nan
    sims = np.dot(x,y)/(x_norm * y_norm)
    dists = 1 - sims
    return dists

## 2-B
Load the data, transpose it so it can be used to cluster the documents, and create a 80-20 train-test split.

In [188]:
data = pd.read_csv('newsgroups5/matrix.txt',header=None)
print(data.shape)
data = data.T
print(data.shape)
data.head()

(9328, 2500)
(2500, 9328)


,0,1,2,3,4,5,6,7,8,9,...,9318,9319,9320,9321,9322,9323,9324,9325,9326,9327
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [189]:
target = pd.read_csv('newsgroups5/classes.txt', skiprows=1, header=None, delimiter=' ', index_col=0)
target.columns = ['Class']
print(target.shape)
target.head()

(2500, 1)


,Class
0,
0,0
1,1
2,1
3,1
4,2


In [190]:
doc_train, doc_test, target_train, target_test = train_test_split(data, target, test_size=0.2, random_state=4378)
print(doc_train.shape)
print(doc_test.shape)

(2000, 9328)
(500, 9328)


Now, convert the training data to TFxIDF.

In [191]:
train_freqs = pd.DataFrame(doc_train.sum(axis=0)).T
train_freqs

,0,1,2,3,4,5,6,7,8,9,...,9318,9319,9320,9321,9322,9323,9324,9325,9326,9327
0,12,6,78,11,26,10,50,4,63,171,...,8,8,3,16,3,5,41,9,4,6


In [192]:
test_freqs = pd.DataFrame(doc_test.sum(axis=0)).T
test_freqs

,0,1,2,3,4,5,6,7,8,9,...,9318,9319,9320,9321,9322,9323,9324,9325,9326,9327
0,4,0,4,1,4,0,6,0,15,41,...,5,8,1,3,4,3,9,2,2,9


In [193]:
train_NMatrix = np.ones(np.shape(doc_train), dtype=float)*doc_train.shape[0]
print(train_NMatrix[:5])

[[2000. 2000. 2000. ... 2000. 2000. 2000.]
 [2000. 2000. 2000. ... 2000. 2000. 2000.]
 [2000. 2000. 2000. ... 2000. 2000. 2000.]
 [2000. 2000. 2000. ... 2000. 2000. 2000.]
 [2000. 2000. 2000. ... 2000. 2000. 2000.]]


In [194]:
test_NMatrix = np.ones(np.shape(doc_test), dtype=float)*doc_test.shape[0]
print(test_NMatrix[:5])

[[500. 500. 500. ... 500. 500. 500.]
 [500. 500. 500. ... 500. 500. 500.]
 [500. 500. 500. ... 500. 500. 500.]
 [500. 500. 500. ... 500. 500. 500.]
 [500. 500. 500. ... 500. 500. 500.]]


In [224]:
train_freqs[train_freqs == 0] = 1
train_IDF = np.log2(np.divide(train_NMatrix, np.array(train_freqs)))
print(train_IDF[:5])

[[7.38082178 8.38082178 4.68038207 ... 7.79585928 8.96578428 8.38082178]
 [7.38082178 8.38082178 4.68038207 ... 7.79585928 8.96578428 8.38082178]
 [7.38082178 8.38082178 4.68038207 ... 7.79585928 8.96578428 8.38082178]
 [7.38082178 8.38082178 4.68038207 ... 7.79585928 8.96578428 8.38082178]
 [7.38082178 8.38082178 4.68038207 ... 7.79585928 8.96578428 8.38082178]]


In [196]:
test_freqs[test_freqs == 0] = 1
test_IDF = np.log2(np.divide(test_NMatrix, np.array(test_freqs)))
print(test_IDF[:5])

[[6.96578428 8.96578428 6.96578428 ... 7.96578428 7.96578428 5.79585928]
 [6.96578428 8.96578428 6.96578428 ... 7.96578428 7.96578428 5.79585928]
 [6.96578428 8.96578428 6.96578428 ... 7.96578428 7.96578428 5.79585928]
 [6.96578428 8.96578428 6.96578428 ... 7.96578428 7.96578428 5.79585928]
 [6.96578428 8.96578428 6.96578428 ... 7.96578428 7.96578428 5.79585928]]


In [197]:
train_TFxIDF = doc_train * train_IDF
print(train_TFxIDF.shape)
train_TFxIDF.head()

(2000, 9328)


,0,1,2,3,4,5,6,7,8,9,...,9318,9319,9320,9321,9322,9323,9324,9325,9326,9327
413,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
869,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [198]:
test_TFxIDF = doc_test * test_IDF
print(test_TFxIDF.shape)
test_TFxIDF.head()

(500, 9328)


,0,1,2,3,4,5,6,7,8,9,...,9318,9319,9320,9321,9322,9323,9324,9325,9326,9327
1317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2184,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [199]:
test_TFxIDF.index[:5]

Index([1317, 1218, 2065, 910, 2184], dtype='int64')

In [200]:
terms = pd.read_csv('newsgroups5/terms.txt',header=None)
terms = np.array(terms)
len(terms)

9328

## 2-C


Analyze the clusters sorted by the top N terms most frequent in each cluster and their centroid weight.

In [201]:
def clusterAnalysis(centroids,clusters,N,terms,NDocs):
    train_idx = NDocs.index
    for c in range(len(centroids)):
        count = np.sum(clusters[:,0]==c) #count the number of items in the cluster
        ids = np.where(clusters[:,0]==c) #get the index of items in cluster
        idx = train_idx[ids] #match the cluster indices to the document data
        print(f"Cluster {c}")
        print(f"Cluster size: {count}")
        cluster_tf = (NDocs.loc[idx] != 0).sum().sort_values(ascending=False)[:N] #sum term frequencies and get top N most frequent
        print(f'Top {N} terms in Cluster {c}')
        print(f"{'term':<10} {'DF':<5} {'% of Ndocs'}")
        for tf in cluster_tf:
            if tf == 0: #empty cluster
                p = 0
            else:
                p = (tf/count)*100
            id = cluster_tf[cluster_tf == tf].index[0] #get index of term
            term = terms[id][0] #get term from data
            print(f"{term:<10} {tf:<5} {p:.6f}")
        print()

Perform KMeans for K values 4 through 8, clustering the TFxIDF data using the cosine_distance function.

In [202]:
for k in range(4,9):
    print('----------------------------------------------------------------------------------')
    print(f'KMeans {k}')
    print('----------------------------------------------------------------------------------')
    train_centroids, train_clusters = kMeans(np.array(train_TFxIDF),k,distMeas=cosine_distance)
    clusterAnalysis(train_centroids,train_clusters,5,terms,doc_train)

----------------------------------------------------------------------------------
KMeans 4
----------------------------------------------------------------------------------
Cluster 0
Cluster size: 442
Top 5 terms in Cluster 0
term       DF    % of Ndocs
subject    442   100.000000
window     272   61.538462
write      188   42.533937
file       153   34.615385
get        149   33.710407

Cluster 1
Cluster size: 1166
Top 5 terms in Cluster 1
term       DF    % of Ndocs
subject    1166  100.000000
write      707   60.634648
on         557   47.770154
articl     532   45.626072
know       402   34.476844

Cluster 2
Cluster size: 386
Top 5 terms in Cluster 2
term       DF    % of Ndocs
subject    386   100.000000
sale       234   60.621762
email      133   34.455959
pleas      124   32.124352
offer      112   29.015544

Cluster 3
Cluster size: 6
Top 5 terms in Cluster 3
term       DF    % of Ndocs
ax         6     100.000000
ax         6     100.000000
ax         6     100.000000
ax     

Using K 4 through 7 produces similar results, where there are 2 or 3 meaningful clusters and the remaining clusters are empty or noise clusters. When K is 8, there are 4 clusters that are produced that seemingly capture key terms for windows, hockey, and for sale, while christian and crypt are, presumably, both captured in the final cluster. Therefore, to get the most meaningful results, we will use K=8.

## 2-D
Compute completeness and homogeneity of the KMeans clustering.

In [220]:
for k in range(4,9):
    print('----------------------------------------------------------------------------------')
    print(f'KMeans {k}')
    kmeans_centroids, kmeans_clusters = kMeans(np.array(train_TFxIDF),k,distMeas=cosine_distance)
    kmeans_labels = kmeans_clusters[:,0].astype(int)
    com = completeness_score(target_train['Class'],kmeans_labels)
    hom = homogeneity_score(target_train['Class'],kmeans_labels)
    print(f'Completeness Score: {com:.3f}')
    print(f'Homogeneity Score: {hom:.3f}')
    print('----------------------------------------------------------------------------------')
    print()

----------------------------------------------------------------------------------
KMeans 4
Completeness Score: 0.758
Homogeneity Score: 0.463
----------------------------------------------------------------------------------

----------------------------------------------------------------------------------
KMeans 5
Completeness Score: 0.758
Homogeneity Score: 0.463
----------------------------------------------------------------------------------

----------------------------------------------------------------------------------
KMeans 6
Completeness Score: 0.756
Homogeneity Score: 0.463
----------------------------------------------------------------------------------

----------------------------------------------------------------------------------
KMeans 7
Completeness Score: 0.620
Homogeneity Score: 0.437
----------------------------------------------------------------------------------

----------------------------------------------------------------------------------
KMeans 8


After multiple runs of computing completeness and homogeneity for different values of K, K=8 produces less variance in each score and consistently has the highest or nearly the highest values for both evaluations. We will continue to use K=8 for the test.

## 2-E
Using the KMeans clusters as class labels, assign the test data to those clusters. Show the predicted label and the cosine similarity to the centroid.

In [222]:
len(kmeans_centroids)

8

In [234]:
test_TFxIDF = np.asarray(test_TFxIDF)
test_TFxIDF[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

For every test item, calculate the cosine distance to each centroid. Assign to the cluster with the lowest distance.

In [245]:
for i in range(len(test_TFxIDF)):
    min = np.inf
    c_distance = []
    for c in range(len(kmeans_centroids)):
        c_distance.append(cosine_distance(test_TFxIDF[i],kmeans_centroids[c]))
    print(f'{i}; Cluster: {np.argmin(c_distance)}; Distance: {np.min(c_distance)}')

0; Cluster: 2; Distance: 0.8451872557823756
1; Cluster: 1; Distance: 0.9222027994616793
2; Cluster: 2; Distance: 0.8524173723259729
3; Cluster: 1; Distance: 0.8485709920450183
4; Cluster: 2; Distance: 0.9111311316610382
5; Cluster: 4; Distance: 0.7744453029058099
6; Cluster: 1; Distance: 0.9485492518284446
7; Cluster: 7; Distance: 0.8951396087015232
8; Cluster: 1; Distance: 0.8060066998688704
9; Cluster: 4; Distance: 0.8190166620184326
10; Cluster: 1; Distance: 0.7632005604545846
11; Cluster: 1; Distance: 0.8522337353496162
12; Cluster: 1; Distance: 0.9088623456238233
13; Cluster: 4; Distance: 0.8296627436302288
14; Cluster: 2; Distance: 0.8149782720006088
15; Cluster: 7; Distance: 0.9508082339041215
16; Cluster: 2; Distance: 0.8964426191785707
17; Cluster: 2; Distance: 0.8589140394226453
18; Cluster: 1; Distance: 0.7748303464387039
19; Cluster: 1; Distance: 0.8703097525904853
20; Cluster: 2; Distance: 0.9663711706111
21; Cluster: 1; Distance: 0.8411332554236615
22; Cluster: 1; Distanc